In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
#!kill -1 -9

In [39]:
(train_ds, val_ds, test_ds), metadata = tfds.load('emnist',
                                                 split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
                                                 with_info=True,
                                                 as_supervised=True)

In [4]:
num_classes = metadata.features['label'].num_classes

In [5]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (28, 28))
    return image, label

In [40]:
train_ds = train_ds.map(preprocess)

In [41]:
val_ds = val_ds.map(preprocess)
test_ds = test_ds = test_ds.map(preprocess)

In [8]:
train_images = tf.data.experimental.cardinality(train_ds).numpy()
print("Number of train images:", train_images)
test_images = tf.data.experimental.cardinality(test_ds).numpy()
print("Number of test images:", test_images)
val_images = tf.data.experimental.cardinality(val_ds).numpy()
print("Number of validation images:", val_images)

Number of train images: 558346
Number of test images: 69793
Number of validation images: 69793


In [42]:
batch_size = 2000
train_ds = train_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)

In [43]:
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)
tval_ds = val_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [44]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(num_classes, activation = 'softmax'))

In [45]:
model.compile(optimizer = 'adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [46]:
epochs = 20

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs = epochs)

Epoch 1/20
280/280 [==============================] - 59s 203ms/step - loss: 0.8994 - accuracy: 0.7422 - val_loss: 0.5505 - val_accuracy: 0.8152
Epoch 2/20
280/280 [==============================] - 56s 201ms/step - loss: 0.4896 - accuracy: 0.8339 - val_loss: 0.4637 - val_accuracy: 0.8374
Epoch 3/20
280/280 [==============================] - 51s 183ms/step - loss: 0.4355 - accuracy: 0.8476 - val_loss: 0.4287 - val_accuracy: 0.8472
Epoch 4/20
280/280 [==============================] - 55s 195ms/step - loss: 0.4074 - accuracy: 0.8549 - val_loss: 0.4111 - val_accuracy: 0.8523
Epoch 5/20
280/280 [==============================] - 52s 185ms/step - loss: 0.3885 - accuracy: 0.8601 - val_loss: 0.4043 - val_accuracy: 0.8552
Epoch 6/20
280/280 [==============================] - 52s 184ms/step - loss: 0.3738 - accuracy: 0.8643 - val_loss: 0.3990 - val_accuracy: 0.8572
Epoch 7/20
280/280 [==============================] - 51s 182ms/step - loss: 0.3612 - accuracy: 0.8679 - val_loss: 0.3935 - val_ac

In [32]:
loss, accuracy = model.evaluate(test_ds)

70/70 [==============================] - 6s 89ms/step - loss: 0.4220 - accuracy: 0.8519
